In [1]:
import json
f = open("../data/US_category_id.json",)

data = json.load(f)

categoryMap = {}
for i in data["items"]:
    categoryMap[i["id"]]  = i["snippet"]["title"]

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("../data/USvideos.csv")
#df.info()

In [3]:
#df.head()

In [4]:
df.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [5]:
df.drop(df.columns[[0,1,5,7,8,9,10,11,12,13,14]], axis = 1, inplace = True)

In [6]:
#df.info()

In [7]:
#remove rows with no values
rows_with_nan = [index for index, row in df.iterrows() if row.isnull().any()]

In [8]:
#print("Rows with NaN: ", len(rows_with_nan))
df.drop(rows_with_nan, inplace = True)

In [9]:
#df.info()
df.drop_duplicates(subset=["title","channel_title","tags","description"], inplace=True)
#df.info()

In [10]:
#borrow kduong for cleaning dataset
import re

def preProcess(data, length):
    data =  re.sub('[^a-zA-Z0-9 \n\...]', ' ', str(data))
    string_encode = str(data).encode("ascii", "ignore")
    string_decode = string_encode.decode()
    data = string_decode
    dataArr = str(data).strip().split(" ")
    for i in  reversed(range(len(dataArr))):
        dataArr[i] = str(dataArr[i]).lower()
        if len(dataArr[i]) == 0:
            del dataArr[i]
        elif len(dataArr[i]) < length:
            del dataArr[i]
    dataArr = ";".join(dataArr)
    return dataArr

In [11]:
videoData = []
for i in range(df.count()[0]):
    current_vid = df.iloc[i]
    row = []
    channel_title = preProcess(current_vid["channel_title"], 0)
    title = preProcess(current_vid["title"], 4)
    tags = preProcess(current_vid["tags"], 4)
    description = preProcess(current_vid["description"], 4)
    category = categoryMap[str(current_vid["category_id"])]
    
    row.append(channel_title)
    row.append(title)
    row.append(tags)
    row.append(description)
    row.append(category)
    videoData.append(row)

In [12]:
import csv  

header = ['Video Title', 'Channel Title', 'Tags', 'Description', 'Category']

with open('video_data.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    writer.writerows(videoData)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier

file = 'video_data.csv'
data = pd.read_csv(file)

In [14]:
data["All"] = data["Tags"] + ";" + data["Video Title"] + ";" +  data["Description"] + ";" +  data["Channel Title"]
vectorizer = CountVectorizer(stop_words= "english")
all_features = vectorizer.fit_transform(data['All'].fillna(' '))

In [15]:
x_train, x_test, y_train, y_test = train_test_split(all_features, data["Category"], test_size = 0.2, random_state=7)

In [16]:
clf = RidgeClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.813500347947112

In [17]:
#borrow kduong for category prediction
def checkCategory(example):
    emptyArr = []
    emptyArr.append(example)
    doc_term_matrix = vectorizer.transform(emptyArr)
    clf.predict(doc_term_matrix)
    class_prob_list = clf.decision_function(doc_term_matrix)[0]
    map = {}
    for i in range(len(class_prob_list)):
        map[str(clf.classes_[i])] = class_prob_list[i]
    sort_classes = sorted(map.items(), key=lambda x: x[1], reverse=True)
    
    print(example + " is most likely category: " + str(sort_classes[0]))
    print(" ")
    print("Probabilities")
    for i in sort_classes:
        print(i[0], i[1])

In [18]:
checkCategory("Lebron James")

Lebron James is most likely category: ('People & Blogs', -0.6764663906199333)
 
Probabilities
People & Blogs -0.6764663906199333
Comedy -0.7038144439106047
Entertainment -0.7549345045905058
Music -0.7660967302827812
News & Politics -0.8140101797525523
Film & Animation -0.8201220906879938
Science & Technology -0.8576223491286075
Howto & Style -0.8629135454433114
Sports -0.9205925285442761
Pets & Animals -0.9397747250689081
Nonprofits & Activism -0.9591450213072201
Education -0.9749585454076392
Autos & Vehicles -0.9799701975629499
Gaming -0.9849542224688872
Travel & Events -0.9861592287242417
Shows -0.9983801355891334
